## Import packages

In [1]:
import pyomo
import pyomo.opt
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
n_time = 72 #test with 3 days

In [3]:
def _auxDictionary(a):
    temp_dictionary = {}
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
    return temp_dictionary
#temp_dict1 = _auxDictionary(loadLimit)

## Data

In [4]:
data = {}
data['Inputs'] = pd.read_csv('Inputs.csv')
data['EVs_Inputs'] = pd.read_csv('EVs_Inputs.csv')
data['alpha'] = pd.read_csv('alpha.csv')
data['PchmaxEV'] = pd.read_csv('PchmaxEV.csv')
data['S'] = pd.read_csv('S.csv')

In [5]:
n_evs = data['EVs_Inputs']['Esoc'].size

## Start Time

In [6]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 14:07:31


## Sets

In [7]:
model = pyo.ConcreteModel()

model.ev = pyo.Set(initialize = np.arange(1, n_evs + 1))
model.t = pyo.Set(initialize = np.arange(1, n_time + 1))


## Parameters

In [8]:
model.EEVmax = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,2]))
model.EEVmin = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,1]))
model.ESoc = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,0]))
model.dT = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,0]))
model.cDA = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,1]))
model.PchmaxEV = pyo.Param(model.ev, model.t, initialize =_auxDictionary(data['PchmaxEV'].to_numpy()))
model.S = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['S'].to_numpy()))
model.alpha = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['alpha'].to_numpy()))
model.RealHour = pyo.Param(model.t, initialize=_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.penalty1 = 1000
model.penalty2 = 1000
model.penalty3 = 0.6
model.Etrip = pyo.Param(model.ev, initialize=_auxDictionary(data['EVs_Inputs'].to_numpy()[:,3]))
model.n = 0.95
model.m = 0.6
model.factor = 1

## Variables

In [9]:
model.PEV = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.EEV = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Etriprelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Eminsocrelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Etripn = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)

## Constraints

In [10]:
def _balance_etripn(m,ev,t): 
    return m.Etripn[ev,t] == m.Etrip[ev]*m.S[ev,t]/(sum([m.S[ev,k] for k in np.arange(1, n_time + 1)]))
model.balance_etripn = pyo.Constraint(model.ev, model.t, rule = _balance_etripn)

def _balance_energy_EVS3(m,ev,t): 
    if t == 24:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] == m.EEVmax[ev]
    return pyo.Constraint.Skip
model.balance_energy_EVS3 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS3)

def _balance_energy_EVS4(m,ev,t): 
    if t == 48:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] == m.EEVmax[ev]
    return pyo.Constraint.Skip
model.balance_energy_EVS4 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS4)

def _balance_energy_EVS5(m,ev,t): 
    if t == 72:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] == m.EEVmax[ev]
    return pyo.Constraint.Skip
model.balance_energy_EVS5 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS5)

def _balance_energy_EVS(m,ev,t): 
    if t == 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.ESoc[ev] + m.PEV[ev,t]*m.dT[t]*m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS)

def _balance_energy_EVS2(m,ev,t): 
    if t > 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.EEV[ev,t-1] + m.PEV[ev,t]*m.dT[t]*m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS2 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS2)

def _power_charging_limit1(m,ev,t): 
    return m.PEV[ev,t] >= 0
model.power_charging_limit1 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit1)

def _power_charging_limit2(m,ev,t): 
    return m.PEV[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_charging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit2)

def _energy_limits_EVS_1(m,ev,t): 
    return m.EEVmin[ev] <= m.EEV[ev,t] + m.Eminsocrelax[ev,t]
model.energy_limits_EVS_1 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_1)

def _energy_limits_EVS_2(m,ev,t): 
    return m.EEV[ev,t] <= m.EEVmax[ev]
model.energy_limits_EVS_2 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_2)  

## Objective function

In [11]:
def _FOag(m):
    return sum([m.EEVmax[ev] - m.EEV[ev,t] + m.Etriprelax[ev,t]*m.penalty1 + m.Eminsocrelax[ev,t]*m.penalty2 for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])  
model.FOag = pyo.Objective(rule = _FOag, sense = pyo.minimize)

## Solve the model

In [12]:
from pyomo.opt import SolverFactory

model.write('res_V4_EC.lp',  io_options={'symbolic_solver_labels': True})

# Create a solver
#opt = pyo.SolverFactory('cbc', executable='C:/Program Files/Cbc-releases.2.10.8-x86_64-w64-mingw64/bin/cbc.exe')

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio129/cplex/bin/x64_win64/cplex.exe')
opt.options['LogFile'] = 'res_V4_EC.log'

#opt = pyo.SolverFactory('ipopt', executable='C:/Program Files/Ipopt-3.11.1-win64-intel13.1/bin/ipopt.exe')
#opt.options['print_level'] = 12
#opt.options['output_file'] = "res_V5_EC.log"

results = opt.solve(model)#, tee=True)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmpg_b27qv7
  Lower bound: 7720339026.437187
  Upper bound: 7720339026.437187
  Number of objectives: 1
  Number of constraints: 636406
  Number of variables: 526681
  Number of nonzeros: 1166012
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 1.63
  Termination condition: optimal
  Termination message: Dual simplex - Optimal\x3a Objective = 7.7203390264e+09
  Error rc: 0
  Time: 4.322736501693726
# ----------------------------------------------------------
#   Solution Information
# --------------

## Objective Function Value

In [13]:
pyo.value(model.FOag)

7720339026.437257

## End Time

In [14]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)
print("Dif: {}".format(datetime.strptime(end_time, "%H:%M:%S") - datetime.strptime(start_time, "%H:%M:%S")))

End Time = 14:09:46
Dif: 0:02:15


## Cost Value

In [15]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())
    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)
    return s

In [16]:
PEV_df = ext_pyomo_vals(model.PEV)
dT_df = ext_pyomo_vals(model.dT)
cDA_df = ext_pyomo_vals(model.cDA)
EEV_df = ext_pyomo_vals(model.EEV)

charge_cost = sum([PEV_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                   for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

print('Charge cost: {}'.format(charge_cost))

Charge cost: 3799029.221786922


## Results

In [17]:
print("Total Charge: {}".format(np.sum(PEV_df.to_numpy()/1000000)))

Total Charge: 53.38876383183451


In [18]:
EEV_df

,1,2,3,4,5,6,7,8,9,10,...,63,64,65,66,67,68,69,70,71,72
1,52800.000000,59640.000000,66480.000000,73320.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,...,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000
2,16200.000000,19620.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,...,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000
3,16200.000000,19620.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,...,20767.399995,20767.399995,20767.399995,20767.399995,20767.399995,20767.399995,20767.399995,20767.399995,20767.399995,20767.399995
4,67079.704236,67079.704236,67079.704236,67079.704236,67079.704236,67079.704236,67079.704236,67079.704236,67079.704236,67079.704236,...,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000
5,45960.000000,45960.000000,45960.000000,45960.000000,45960.000000,45960.000000,45960.000000,45960.000000,45960.000000,45960.000000,...,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000,76600.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,55245.936702,55245.936702,55245.936702,55245.936702,55245.936702,55245.936702,55245.936702,55245.936702,55245.936702,55245.936702,...,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000
1460,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,...,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000
1461,41340.000000,48180.000000,55020.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,...,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000,57500.000000
1462,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,...,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000,21300.000000


In [19]:
PEV_df

,1,2,3,4,5,6,7,8,9,10,...,63,64,65,66,67,68,69,70,71,72
1,7200.000000,7200.0,7200.000000,7200.000000,3452.631579,0.0,0.0,0.0,0.0,1901.392611,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,1901.392611
2,3600.000000,3600.0,1768.421053,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
3,3600.000000,3600.0,1768.421053,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,3600.000000
4,22000.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,21.363962,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
5,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,919.504921,0.0,0.0,0.0,0.0,919.504921,0.0,919.504921,0.0,919.504921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,22000.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,162.171893,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
1460,9067.957693,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,99.536641,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
1461,7200.000000,7200.0,7200.000000,2610.526316,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
1462,8968.421053,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [20]:
PEV_df.sum()

1     9.242645e+06
2     3.470611e+06
3     2.815705e+06
4     1.777347e+06
5     1.023242e+06
          ...     
68    2.203442e+05
69    1.589199e+05
70    3.195400e+05
71    1.257768e+06
72    1.729723e+06
Length: 72, dtype: float64

In [21]:
Etriprelax_df = ext_pyomo_vals(model.Etriprelax)

In [22]:
Etriprelax_df.sum()

1     0.000000e+00
2     0.000000e+00
3     0.000000e+00
4     0.000000e+00
5     0.000000e+00
          ...     
68    7.887033e+03
69    4.899160e+04
70    0.000000e+00
71    0.000000e+00
72    1.569822e+06
Length: 72, dtype: float64

In [23]:
Eminsocrelax_df = ext_pyomo_vals(model.Eminsocrelax)

In [24]:
Eminsocrelax_df.sum()

1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
     ... 
68    0.0
69    0.0
70    0.0
71    0.0
72    0.0
Length: 72, dtype: float64

In [25]:
Etripn_df = ext_pyomo_vals(model.Etripn)

In [26]:
Etripn_df.sum()

1     1.729799e+04
2     0.000000e+00
3     0.000000e+00
4     0.000000e+00
5     0.000000e+00
          ...     
68    6.291725e+05
69    1.044228e+06
70    7.452049e+05
71    1.034969e+06
72    8.203071e+05
Length: 72, dtype: float64

## Save results in csv files

In [27]:
import os 
folder = 'results_Dumb_' + str(n_evs)

if not os.path.exists(folder):
    os.makedirs(folder)
    
EEV_df.to_csv(folder + '/EEV.csv')
PEV_df.to_csv(folder + '/PEV.csv')
PEV_df.sum().to_csv(folder + '/PEV_h.csv')

Etriprelax_df.to_csv(folder + '/Etriprelax.csv')
Etriprelax_df.sum().to_csv(folder + '/Etriprelax_h.csv')

Eminsocrelax_df.to_csv(folder + '/Eminsocrelax.csv')
Eminsocrelax_df.sum().to_csv(folder + '/Eminsocrelax_h.csv')

Etripn_df.to_csv(folder + '/Etripn.csv')
Etripn_df.sum().to_csv(folder + '/Etripn_h.csv')